In [800]:
import pickle
from sklearn import preprocessing
import numpy as np
import math
from scipy.stats import norm
from sklearn import datasets
import csv
import warnings
from statistics import NormalDist
warnings.filterwarnings("ignore")

In [849]:
x_train=list()
y_train=list()
z=list()
with open('/Users/chengyu/Downloads/20220108_sentiment_feature_fixed.csv', newline='') as csvfile:
     # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
    rows = csv.DictReader(csvfile)

    # 以迴圈輸出指定欄位
    for row in rows:
        x_train.append([row['gun_pos'],row['gun_neg'],
                        row['teamwork_pos'],row['teamwork_neg'],
                        row['loot_pos'],row['loot_neg'],
                        row['price_pos'],row['price_neg'],
                        row['cheat_pos'],row['cheat_neg'],
                        row['optimization_pos'],row['optimization_neg'],
                        row['graphic_pos'],row['graphic_neg']])
        y_train.append(row['recommended'])

with open('/Users/chengyu/Downloads/20220108_sentiment_feature_final.csv', newline='') as csvfile:
     # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
    rows = csv.DictReader(csvfile)
    for row in rows:
        z.append(row['auther.num_reivews'])
z = np.array(z,dtype=float)
z=(z - np.mean(z)) / np.std(z)
print(z)
x_train =np.array(x_train,dtype=float)
y_train = np.array(y_train,dtype=float)

[-0.3494282   4.3697517  -0.26038707 ... -0.30490764 -0.3494282
 -0.30490764]


In [850]:
def n_one(a):
    return a.reshape((len(X),1))

In [862]:
class ProbitRegression:

    def __init__(self, y, X, beta, z, gamma=0):
        self.X, self.y, self.beta, self.z = X, y, beta, z #beta是係數 z為評論：為每個評論的作者的總評論數
        self.n, self.k = X.shape
        self.gamma = gamma
    
    def exp(self):
        return np.exp(-1*self.z*self.gamma)
    def cdf(self):  # cumulative distribution function
        exp=self.exp()
        return  norm.cdf(self.X.dot(self.beta) * exp)# @是矩陣相乘

    def pdf(self):  # probability density function
        exp=self.exp()
        #print(self.X.dot(self.beta).shape)
        return norm.pdf(self.X.dot(self.beta) * exp)

    def logL(self): #loss
        cdf = self.cdf()
        return np.sum(self.y * np.log(cdf) + (1 - self.y) * np.log(1 - cdf))    

    def G(self):  # gradient
        X = self.X
        y = self.y
        exp = self.exp()
        gamma = self.gamma
        z = self.z
        beta = self.beta
        cdf = self.cdf()
        pdf = self.pdf()
        print(np.where(cdf == 1)[0])
        #you will get all beta j's data value since Differential
        return np.append(
                        np.sum(
                                y * ((X.T*pdf*exp) / cdf) + 
                                (1-y) * (-1) * ((X.T*(-pdf)*exp) / (1-cdf))
                                ,axis=1
                              ), 
                        np.sum(
                               y * ((pdf*exp*(X.dot(beta))*-z)/cdf) +
                               (1-y) * (-1) * (pdf*exp*(X.dot(beta))*(-z)/(1-cdf))
                               , axis=0
                              )
                        )
    

    def H(self): #hessian
        X = self.X
        y=self.y
        exp = self.exp()
        gamma = self.gamma
        z = self.z
        beta = self.beta
        cdf = self.cdf()
        pdf = self.pdf()
        #print(X.T.shape)
        H11=list() #14*14
        H01=list() #14*1
        H10=list() #1*14
        H00=list() #1*1
        for i in range(X.T.shape[0]): 
            temp = list()
            for j in range (X.T.shape[0]):
                temp.append(
                            np.sum(
                                    y *(X.T[i]*exp)*(X.T[j]*exp)*((-1*pdf*pdf)/(cdf*cdf)- (pdf*(X.dot(beta)*exp))/cdf)+
                                    (1-y)*(-1)*(X.T[i]*exp)*(X.T[j]*exp)*((pdf*pdf)/(1-(cdf*cdf)) - (pdf*(X.dot(beta)*exp))/(1-cdf))
                                  )
                            )
            H11.append(temp)
        
        for i in range(X.T.shape[0]): 
            H01.append(
                       np.sum(
                                y*(-1*exp*z)*(((pdf/cdf)*X.T[i])+(X.T[i]*exp)*((-1*pdf*pdf)/(cdf*cdf) - (pdf*(X.dot(beta)*exp))/cdf))+
                                (1-y)*(exp*z)*(((pdf/(1-cdf))*X.T[i])+(X.T[i]*exp)*((pdf*pdf)/(1-(cdf*cdf)) - (pdf*(X.dot(beta)*exp))/(1-cdf)))
                             )
                      )
        for i in range(X.T.shape[0]): 
            H10.append(
                       np.sum(
                               y *X.T[i] *(-1*exp*z)*((pdf/cdf)-((X.dot(beta)*exp)*((pdf*pdf)/(cdf*cdf) + (pdf/cdf)*(X.dot(beta)*exp))))+
                               (1-y) * (-1*X.T[i]) * (-1*exp*z)* ((pdf/(1-cdf)) + ((X.dot(beta)*exp)*((pdf*pdf)/(cdf*cdf) + (pdf/cdf)*(X.dot(beta)*exp))))
                             )
                    )
        H00.append(
                    np.sum(
                            y*(-1*z*X.dot(beta)) * (-1*exp*z)*((pdf/cdf)-((X.dot(beta)*exp)*((pdf*pdf)/(cdf*cdf) + (pdf/cdf)*(X.dot(beta)*exp)))) +
                            (1-y)*(z*X.dot(beta)) * (-1*exp*z)* ((pdf/(1-cdf)) + ((X.dot(beta)*exp)*((pdf*pdf)/(cdf*cdf) + (pdf/cdf)*(X.dot(beta)*exp))))
                          )
                  )

        H11=np.array(H11)
        H01=np.array(H01).reshape(1,14)
        H10=np.array(H10).reshape(1,14)
        H00=np.array(H00).reshape(1,1)
        H = np.append(H11,H01,axis=0)
        temp_H = np.append(H10,H00).reshape(1,15)
        H = np.append(H,temp_H.T,axis=1)
        
        return H

In [863]:
def newton_raphson(model, tol=1e-3, max_iter=1000, display=True):

    i = 0
    error = 100  # Initial error value

    # Print header of output
    if display:
        header = f'{"Iteration_k":<13}{"Log-likelihood":<16}{"θ":<60}'
        print(header)
        print("-" * len(header))

    # While loop runs while any value in error is greater
    # than the tolerance until max iterations are reached
    while np.any(error > tol) and i < max_iter:
        H, G = model.H(), model.G()
        #print(H)
        #print(G)
        print('before')
        print(model.beta)
        print(model.gamma)
        param = np.append(model.beta,model.gamma)
        
        
        beta_new = param - (np.linalg.inv(H).dot(G))
        error = np.linalg.norm(beta_new - param)
        model.beta = np.array(beta_new[:14])
        model.gamma = np.array(beta_new[-1])
        print('after')
        print(model.beta)
        print(model.gamma)
        # Print iterations
        if display:
            beta_list = [f'{t:.3}' for t in list(model.beta.flatten())]
            update = f'{i:<13}{model.logL():<16.8}{beta_list}'
            #print(update)

        i += 1

    print(f'Number of iterations: {i}')
    print(f'β_hat = {model.beta.flatten()}')

    # Return a flat array for β (instead of a k_by_1 column vector)
    return model.beta,model.gamma

In [864]:
model = ProbitRegression(y_train, x_train, np.zeros(len(x_train[0])),z,1)

In [865]:
# !!!!!!!!!
newton_raphson(model)

Iteration_k  Log-likelihood  θ                                                           
-----------------------------------------------------------------------------------------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[]
before
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1
after
[ 1.80078727  0.34804575  2.14644351  0.45175476  0.49637737  0.15492852
  1.65560066 -1.65137947  1.18600136 -0.46598375  1.50372907  0.24801116


(array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan]),
 array(nan))

In [847]:
print(model.G().shape)

[]
(15,)


In [ ]:
print(model.H())